In [ ]:
# | default_exp pointNet

# Imports

In [ ]:
# | export
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class TNet(nn.Module):
    def __init__(self, k=3):
        super(TNet, self).__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)

        self.bn1 = nn.InstanceNorm1d(64, affine=True)
        self.bn2 = nn.InstanceNorm1d(128, affine=True)
        self.bn3 = nn.InstanceNorm1d(1024, affine=True)
        self.bn4 = nn.Identity()
        self.bn5 = nn.Identity()
        # self.bn4 = nn.InstanceNorm1d(512, affine=True)
        # self.bn5 = nn.InstanceNorm1d(256, affine=True)

    def forward(self, x):
        B, K, N = x.size()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2)[0]
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = torch.eye(self.k).flatten().to(x.device)
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

In [ ]:
class PointNetSeg(nn.Module):
    def __init__(self, input_size, num_classes):
        super(PointNetSeg, self).__init__()
        self.input_transform = TNet(k=3)

        self.conv1 = nn.Conv1d(input_size, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)

        self.bn1 = nn.InstanceNorm1d(64, affine=True)
        self.bn2 = nn.InstanceNorm1d(128, affine=True)
        self.bn3 = nn.InstanceNorm1d(1024, affine=True)

        self.fc1 = nn.Conv1d(1088, 512, 1)
        self.fc2 = nn.Conv1d(512, 256, 1)
        self.fc3 = nn.Conv1d(256, num_classes, 1)

        self.bn4 = nn.InstanceNorm1d(512)
        self.bn5 = nn.InstanceNorm1d(256)

        # self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        B, N, _ = x.size()
        x = x.transpose(2, 1)  # B x 3 x N

        trans = self.input_transform(x)
        x = torch.bmm(trans, x)

        x = F.relu(self.bn1(self.conv1(x)))  # B x 64 x N
        x1 = x
        x = F.relu(self.bn2(self.conv2(x)))  # B x 128 x N
        x = F.relu(self.bn3(self.conv3(x)))  # B x 1024 x N

        x = torch.max(x, 2, keepdim=True)[0]  # B x 1024 x 1
        x = x.repeat(1, 1, N)  # B x 1024 x N

        x = torch.cat([x1, x], 1)  # B x 1088 x N

        x = F.relu(self.bn4(self.fc1(x)))  # B x 512 x N
        # x = F.relu(self.bn5(self.dropout(self.fc2(x))))  # B x 256 x N
        x = F.relu(self.bn5(self.fc2(x)))  # B x 256 x N
        x = self.fc3(x)  # B x num_classes x N

        x = x.transpose(2, 1).contiguous()  # B x N x num_classes
        return x

# copia

In [ ]:
# | export
class TNet(nn.Module):
    def __init__(self, k=3):
        super(TNet, self).__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)

        # self.bn1 = nn.BatchNorm1d(64)
        # self.bn2 = nn.BatchNorm1d(128)
        # self.bn3 = nn.BatchNorm1d(1024)
        self.bn1 = nn.InstanceNorm1d(64)
        self.bn2 = nn.InstanceNorm1d(128)
        self.bn3 = nn.InstanceNorm1d(1024)
        # self.bn4 = nn.BatchNorm1d(512)
        # self.bn5 = nn.BatchNorm1d(256)
        self.bn4 = nn.LayerNorm(512)
        self.bn5 = nn.LayerNorm(256)

    def forward(self, x):
        B, K, N = x.size()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2)[0]
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = torch.eye(self.k, device=x.device).flatten().unsqueeze(0).repeat(B, 1)
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

In [ ]:
# | export
class PointNetEncoder(nn.Module):
    def __init__(self, global_feat=True, feature_transform=False, channel=9):
        super(PointNetEncoder, self).__init__()
        self.stn = TNet(k=3)
        self.conv1 = nn.Conv1d(channel, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.InstanceNorm1d(64)
        self.bn2 = nn.InstanceNorm1d(128)
        self.bn3 = nn.InstanceNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = TNet(k=64)

    def forward(self, x):
        B, C, N = x.size()
        # Input transform only on first 3 dims (xyz)
        trans = self.stn(x[:, :3, :])
        x_xyz = x[:, :3, :]
        x_rest = x[:, 3:, :] if C > 3 else None
        x_xyz = torch.bmm(trans, x_xyz)
        if x_rest is not None:
            x = torch.cat([x_xyz, x_rest], dim=1)
        else:
            x = x_xyz
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = torch.bmm(trans_feat, x)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, N)
            return torch.cat([x, pointfeat], 1), trans, trans_feat

In [ ]:
# | export
class PointNetSeg(nn.Module):
    def __init__(self, num_classes, input_channels=9, feature_transform=True):
        super(PointNetSeg, self).__init__()
        self.num_classes = num_classes
        self.feat = PointNetEncoder(
            global_feat=False,
            feature_transform=feature_transform,
            channel=input_channels,
        )
        self.conv1 = nn.Conv1d(1088, 512, 1)
        self.conv2 = nn.Conv1d(512, 256, 1)
        self.conv3 = nn.Conv1d(256, 128, 1)
        self.conv4 = nn.Conv1d(128, self.num_classes, 1)
        self.bn1 = nn.InstanceNorm1d(512)
        self.bn2 = nn.InstanceNorm1d(256)
        self.bn3 = nn.InstanceNorm1d(128)

    def forward(self, x):
        # Espera entrada: (B, N, C) -> (B, C, N)
        x = x.transpose(1, 2)  # (B, C, N)
        # if x.shape[1] < x.shape[2]:
        #     x = x.transpose(1, 2)  # (B, C, N)

        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2, 1).contiguous()  # (B, N, num_classes)
        # falta aplicar softmax e view do codigo copiado
        return x, trans_feat

In [ ]:
# | export
def feature_transform_regulaizer(trans):
    d = trans.size()[1]
    I = torch.eye(d, device=trans.device)[None, :, :]
    loss = torch.mean(
        torch.norm(torch.bmm(trans, trans.transpose(2, 1)) - I, dim=(1, 2))
    )
    return loss